<h2>解析库(XPath BeautifulSoup PyQuery)的使用</h2>

提取页面信息时使用正则表达式比较烦琐，写错可能导致匹配失败（使用正则表达式提取页面信息不方便）。

对于网页的节点来说，它可以定义 id、class 或其他属性。而且节点之间还有层次关系，

在网页中可以通过 XPath 或 CSS 选择器来定位一个或多个节点。

实现这个操作可以使用的解析库非常多，其中比较强大的库有 lxml、Beautiful Soup、pyquery 等。

<h3>使用 XPath</h3>

(XPath, XML Path Language)，即 XML 路径语言，它是一门在 XML 文档中查找信息的语言。它最初是用来搜寻 XML 文档的，但是它同样适用于 HTML 文档的搜索。


XPath 于 1999 年 11 月 16 日成为 W3C 标准，它被设计为供 XSLT、XPointer 以及其他 XML 解析软件使用，更多的文档可以访问其官方网站：[https://www.w3.org/TR/xpath/](https://www.w3.org/TR/xpath/)

**XPath 常用规则**

```
表　达　式	描　　述
nodename	选取此节点的所有子节点
/	从当前节点选取直接子节点
//	从当前节点选取子孙节点
.	选取当前节点
..	选取当前节点的父节点
@	选取属性
```
XPath 的常用匹配规则，示例：`//title[@lang='eng']`，它代表选择所有名称为 title，同时属性 lang 的值为 eng 的节点。

通过实例来感受一下使用 XPath 来对网页进行解析的过程，相关代码如下：
> 注意：HTML 文本中的最后一个 `li` 节点是没有闭合的，但是 etree 模块可以自动修正 HTML 文本。（并且还自动添加了 `body`、`html` 节点）
> 
> 调用 `tostring` 方法即可输出修正后的 HTML 代码，但是结果是 `bytes` 类型。这里利用 `decode` 方法将其转成 `str` 类型

In [1]:
from lxml import etree
text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = etree.tostring(html)
print(result.decode('utf-8'))

<html><body><div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </li></ul>
 </div>
</body></html>


另外，也可以直接读取文本文件进行解析，示例如下：
```python
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = etree.tostring(html)
print(result.decode('utf-8'))
```
> 会多出一个 `DOCTYPE` 的声明

**所有节点**

一般会用 `//` 开头的 XPath 规则来选取所有符合要求的节点。这里以前面的 HTML 文本为例，如果要选取所有节点，可以这样实现：
```python
from lxml import etree
html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//*')
print(result)
```

使用 `*` 代表匹配所有节点，也就是整个 HTML 文本中的所有节点都会被获取。

返回形式是一个列表，每个元素是 Element 类型，其后跟了节点的名称，如 `html、body、div、ul、li、a` 等，所有节点都包含在列表中了。

想获取所有 li 节点：`result = html.xpath('//li')`

**子节点**

通过 `/` 或 `//` 即可查找元素的子节点或子孙节点。

假如想选择 li 节点的所有直接 a 子节点，可以这样实现：

In [2]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li/a')  # 等价于 `//li//a`
print(result)

[<Element a at 0x1519a62c1c0>, <Element a at 0x1519a62c280>, <Element a at 0x1519a62c440>, <Element a at 0x1519a62c900>, <Element a at 0x1519b844600>]


**父节点**

假如知道了子节点，可以用`..`来查找父节点呢。

比如，现在首先选中 `href` 属性为 `link4.html` 的 `a` 节点，然后再获取其父节点，然后再获取其 `class` 属性，

相关代码：`result = html.xpath('//a[@href="link4.html"]/../@class')`

也可以通过 `parent::` 来获取父节点，代码：`result = html.xpath('//a[@href="link4.html"]/parent::*/@class')`

**文本获取**

用 XPath 中的 `text` 方法获取节点中的文本，尝试获取前面 li 节点中的文本，相关代码：
`result = html.xpath('//li[@class="item-0"]/text()')` 或者 `result = html.xpath('//li[@class="item-0"]//text()')`

**属性获取**

节点属性用 `@`符号就可以获取。例如，想获取所有 li 节点下所有 a 节点的 href 属性，代码：`result = html.xpath('//li/a/@href')`

**属性多值匹配**

有时候，某些节点的某个属性可能有多个值，例如：
> 这里 HTML 文本中 li 节点的 class 属性有两个值 li 和 li-first，此时如果还想用之前的属性匹配获取，就无法匹配了，此时的运行结果 —— `[]`

In [3]:
from lxml import etree  
text = '''  
<li class="li li-first"><a href="link.html">first item</a></li> 
'''  
html = etree.HTML(text)  
result = html.xpath('//li[@class="li"]/a/text()')  
print(result)

[]


这时就需要用 `contains` 方法了，代码可以改写如下：

In [4]:
from lxml import etree  
text = '''  
<li class="li li-first"><a href="link.html">first item</a></li>  
'''  
html = etree.HTML(text)  
result = html.xpath('//li[contains(@class, "li")]/a/text()')  
print(result)

['first item']


**多属性匹配**

根据多个属性确定一个节点，需要同时匹配多个属性。此时可以使用运算符 `and` 来连接，示例如下：

In [5]:
from lxml import etree  
text = '''  
<li class="li li-first" name="item"><a href="link.html">first item</a></li>
'''  
html = etree.HTML(text)  
result = html.xpath('//li[contains(@class, "li") and @name="item"]/a/text()')  
print(result)

['first item']


运算符及其介绍
```
运算符	描　　述	实　　例	返　回　值
or	或	age=19 or age=20	如果 age 是 19，则返回 true。如果 age 是 21，则返回 false
and	与	age>19 and age<21	如果 age 是 20，则返回 true。如果 age 是 18，则返回 false
mod	计算除法的余数	5 mod 2	1
计算两个节点集	//book
+	加法	6 + 4	10
-	减法	6 - 4	2
*	乘法	6 * 4	24
div	除法	8 div 4	2
=	等于	age=19	如果 age 是 19，则返回 true。如果 age 是 20，则返回 false
!=	不等于	age!=19	如果 age 是 18，则返回 true。如果 age 是 19，则返回 false
<	小于	age<19	如果 age 是 18，则返回 true。如果 age 是 19，则返回 false
<=	小于或等于	age<=19	如果 age 是 19，则返回 true。如果 age 是 20，则返回 false
>	大于	age>19	如果 age 是 20，则返回 true。如果 age 是 19，则返回 false
>=	大于或等于	age>=19	如果 age 是 19，则返回 true。如果 age 是 18，则返回 false
```

**按序选择**

在选择的时候某些属性可能同时匹配了多个节点，但是只想要其中的某个节点，如第二个节点或者最后一个节点，可以利用**中括号传入索引的方法获取特定次序的节点**，示例如下：

In [6]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html">first item</a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li[1]/a/text()')
print(result)
result = html.xpath('//li[last()]/a/text()')
print(result)
result = html.xpath('//li[position()<3]/a/text()')
print(result)
result = html.xpath('//li[last()-2]/a/text()')
print(result)

['first item']
['fifth item']
['first item', 'second item']
['third item']


**节点轴选择**

XPath 提供了很多节点轴选择方法，包括获取子元素、兄弟元素、父元素、祖先元素等，示例如下：

In [7]:
from lxml import etree

text = '''
<div>
    <ul>
         <li class="item-0"><a href="link1.html"><span>first item</span></a></li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-inactive"><a href="link3.html">third item</a></li>
         <li class="item-1"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a>
     </ul>
 </div>
'''
html = etree.HTML(text)
result = html.xpath('//li[1]/ancestor::*')
print(result)
result = html.xpath('//li[1]/ancestor::div')
print(result)
result = html.xpath('//li[1]/attribute::*')
print(result)
result = html.xpath('//li[1]/child::a[@href="link1.html"]')
print(result)
result = html.xpath('//li[1]/descendant::span')
print(result)
result = html.xpath('//li[1]/following::*[2]')
print(result)
result = html.xpath('//li[1]/following-sibling::*')
print(result)

[<Element html at 0x15198c30280>, <Element body at 0x15198c2b2c0>, <Element div at 0x15198c30200>, <Element ul at 0x15198c30540>]
[<Element div at 0x15198c30200>]
['item-0']
[<Element a at 0x15198c306c0>]
[<Element span at 0x15198c30200>]
[<Element a at 0x15198c304c0>]
[<Element li at 0x15198c305c0>, <Element li at 0x15198c30400>, <Element li at 0x15198c30380>, <Element li at 0x15198c30680>]


<h3>使用 Beautiful Soup</h3>

BeautifulSoup 是 Python 的一个 HTML 或 XML 的解析库，可以用它来方便地从网页中提取数据，官方的解释如下：
> BeautifulSoup 提供一些简单的、Python 式的函数用来处理导航、搜索、修改分析树等功能。它是一个工具箱，通过解析文档为用户提供需要抓取的数据，因为简单，所以不需要多少代码就可以写出一个完整的应用程序。 BeautifulSoup 自动将输入文档转换为 Unicode 编码，输出文档转换为 utf-8 编码。你不需要考虑编码方式，除非文档没有指定一个编码方式，这时你仅仅需要说明一下原始编码方式就可以了。 BeautifulSoup 已成为和 lxml、html5lib 一样出色的 Python 解释器，为用户灵活地提供不同的解析策略或强劲的速度。

> 请确保已经正确安装好了 `Beautiful Soup` 和 `lxml`

**解析器**

Beautiful Soup 在解析时实际上依赖解析器，它除了支持 Python 标准库中的 HTML 解析器外，还支持一些第三方解析器（比如 lxml）。

```
解析器	               使用方法	                               优势                               	          劣势
Python 标准库	BeautifulSoup(markup, "html.parser")	Python 的内置标准库、执行速度适中 、文档容错能力强	        Python 2.7.3 or 3.2.2) 前的版本中文容错能力差
LXML HTML 解析器	BeautifulSoup(markup, "lxml")	        速度快、文档容错能力强	                                需要安装 C 语言库
LXML XML 解析器	BeautifulSoup(markup, "xml")	        速度快、唯一支持 XML 的解析器	                        需要安装 C 语言库
html5lib	    BeautifulSoup(markup, "html5lib")	    最好的容错性、以浏览器的方式解析文档、生成 HTML5 格式的文档	速度慢、不依赖外部扩展
```

In [1]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('<p>Hello</p>', 'lxml')
print(soup.p.string)

Hello


**基本使用**

In [2]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
# 把要解析的字符串以标准的缩进格式输出。这里需要注意的是，输出结果里面包含 body 和 html 节点
# 对于不标准的 HTML 字符串, BeautifulSoup 可以自动更正格式。
print(soup.prettify())
print(soup.title.string)

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title" name="dromouse">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    <!-- Elsie -->
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>
The Dormouse's story


**节点选择器**

直接调用节点的名称就可以选择节点元素，再调用 string 属性就可以得到节点内的文本了，这种选择方式速度非常快。如果单个节点结构层次非常清晰，可以选用这种方式来解析。

In [6]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.title)
print(type(soup.title))
print(soup.title.string)
print(soup.head)
print(soup.p)
# 利用 name 属性获取节点的名称
print(soup.title.name)
# 每个节点可能有多个属性，比如 id 和 class 等，选择这个节点元素后，可以调用 attrs 获取所有属性：
print(soup.p.attrs)
print(soup.p.attrs['name'])
print(soup.p['name'])  # 与上一行代码等价

<title>The Dormouse's story</title>
<class 'bs4.element.Tag'>
The Dormouse's story
<head><title>The Dormouse's story</title></head>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
title
{'class': ['title'], 'name': 'dromouse'}
dromouse
dromouse


**嵌套选择**

返回结果是 `bs4.element.Tag` 类型的同样可以继续调用节点进行下一步的选择。

比如，获取了 head 节点元素，可以继续调用 head 来选取其内部的 head 节点元素：

In [7]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.head.title)
print(type(soup.head.title))
print(soup.head.title.string)

<title>The Dormouse's story</title>
<class 'bs4.element.Tag'>
The Dormouse's story


**关联选择**

在做选择的时候，有时候不能做到一步就选到想要的节点元素，需要先选中某一个节点元素，然后以它为基准再选择它的子节点、父节点、兄弟节点等

选取节点元素之后，如果想要获取它的直接子节点，可以调用 `contents` 属性（或者调用 `children` 属性得到相应的结果），示例如下：

In [9]:
html = """
<html>
    <head>
        <title>The Dormouse's story</title>
    </head>
    <body>
        <p class="story">
            Once upon a time there were three little sisters; and their names were
            <a href="http://example.com/elsie" class="sister" id="link1">
                <span>Elsie</span>
            </a>
            <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> 
            and
            <a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>
            and they lived at the bottom of a well.
        </p>
        <p class="story">...</p>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.p.contents)

print(soup.p.children)
for i, child in enumerate(soup.p.children):
    print(i, child)

['\n            Once upon a time there were three little sisters; and their names were\n            ', <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>, '\n', <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, ' \n            and\n            ', <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>, '\n            and they lived at the bottom of a well.\n        ']
0 
            Once upon a time there were three little sisters; and their names were
            
1 <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>
2 

3 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>
4  
            and
            
5 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
6 
            and they lived at the bottom of a well.
        


如果要得到所有的子孙节点的话，可以调用 `descendants` 属性（递归查询所有子节点，得到所有的子孙节点）：

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.p.descendants)
for i, child in enumerate(soup.p.descendants):
    print(i, child)

<generator object Tag.descendants at 0x000002049469FF20>
0 
            Once upon a time there were three little sisters; and their names were
            
1 <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>
2 

3 <span>Elsie</span>
4 Elsie
5 

6 

7 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>
8 Lacie
9  
            and
            
10 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
11 Tillie
12 
            and they lived at the bottom of a well.
        


**父节点和祖先节点**

如果要获取某个节点元素的父节点，可以调用 `parent` 属性（获取所有的祖先节点，可以调用 `parents` 属性）：

In [13]:
html = """
<html>
    <head>
        <title>The Dormouse's story</title>
    </head>
    <body>
        <p class="story">
            Once upon a time there were three little sisters; and their names were
            <a href="http://example.com/elsie" class="sister" id="link1">
                <span>Elsie</span>
            </a>
        </p>
        <p class="story">...</p>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.a.parent)

print(type(soup.a.parents))  # generator
print(list(enumerate(soup.a.parents)))

<p class="story">
            Once upon a time there were three little sisters; and their names were
            <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>
</p>
<class 'generator'>
[(0, <p class="story">
            Once upon a time there were three little sisters; and their names were
            <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>
</p>), (1, <body>
<p class="story">
            Once upon a time there were three little sisters; and their names were
            <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>
</p>
<p class="story">...</p>
</body>), (2, <html>
<head>
<title>The Dormouse's story</title>
</head>
<body>
<p class="story">
            Once upon a time there were three little sisters; and their names were
            <a class="sister" href="http://example.com/elsie" id="link1">
<span>Elsie</span>
</a>
</p>
<p class="story">...</p>
</body></html>), 

**兄弟节点**

In [14]:
html = """
<html>
    <body>
        <p class="story">
            Once upon a time there were three little sisters; and their names were
            <a href="http://example.com/elsie" class="sister" id="link1">
                <span>Elsie</span>
            </a>
            Hello
            <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> 
            and
            <a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>
            and they lived at the bottom of a well.
        </p>
"""
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print('Next Sibling', soup.a.next_sibling)
print('Prev Sibling', soup.a.previous_sibling)

print('Next Siblings', list(enumerate(soup.a.next_siblings)))
print('Prev Siblings', list(enumerate(soup.a.previous_siblings)))

Next Sibling 
            Hello
            
Prev Sibling 
            Once upon a time there were three little sisters; and their names were
            
Next Siblings [(0, '\n            Hello\n            '), (1, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>), (2, ' \n            and\n            '), (3, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>), (4, '\n            and they lived at the bottom of a well.\n        ')]
Prev Siblings [(0, '\n            Once upon a time there were three little sisters; and their names were\n            ')]


**方法选择器**

前面所讲的选择方法都是通过属性来选择的，这种方法非常快，但是如果进行比较复杂的选择的话，它就比较烦琐了。

Beautiful Soup 还提供了一些查询方法，比如 `find_all` 和 `find` 等

API：`find_all(name , attrs , recursive , text , **kwargs)`

In [22]:
html='''
<div class="panel">
    <div class="panel-heading">
        <h4>Hello</h4>
    </div>
    <div class="panel-body">
        <ul class="list" id="list-1">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
            <li class="element">Jay</li>
        </ul>
        <ul class="list list-small" id="list-2">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
        </ul>
    </div>
</div>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
# 根据节点名来查询元素，使用`name`参数
print(soup.find_all(name='ul'))
print(type(soup.find_all(name='ul')[0]))

for ul in soup.find_all(name='ul'):
    print(ul.find_all(name='li'))
    for li in ul.find_all(name='li'):
        print(li.string)
        
print("------------------------------------")
# 传入一些属性来进行查询，使用`attrs`参数
print(soup.find_all(attrs={'id': 'list-1'}))
print(soup.find_all(attrs={'class': 'element'}))
# 等价方式
print(soup.find_all(id='list-1'))
print(soup.find_all(class_='element'))

[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>, <ul class="list list-small" id="list-2">
<li class="element">Foo</li>
<li class="element">Bar</li>
</ul>]
<class 'bs4.element.Tag'>
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>]
Foo
Bar
Jay
[<li class="element">Foo</li>, <li class="element">Bar</li>]
Foo
Bar
------------------------------------
[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]
[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="e

`text` 参数可**用来匹配节点的文本**，传入的形式可以是字符串，可以是正则表达式对象，示例如下：

In [23]:
import re
html='''
<div class="panel">
    <div class="panel-body">
        <a>Hello, this is a link</a>
        <a>Hello, this is a link, too</a>
    </div>
</div>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.find_all(text=re.compile('link')))

['Hello, this is a link', 'Hello, this is a link, too']


`find` 方法返回的是单个元素，也就是第一个匹配的元素，而 `find_all` 返回的是所有匹配的元素组成的列表。示例如下：
> 返回结果不再是列表形式，而是第一个匹配的节点元素，类型依然是 Tag 类型

In [24]:
html='''
<div class="panel">
    <div class="panel-heading">
        <h4>Hello</h4>
    </div>
    <div class="panel-body">
        <ul class="list" id="list-1">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
            <li class="element">Jay</li>
        </ul>
        <ul class="list list-small" id="list-2">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
        </ul>
    </div>
</div>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.find(name='ul'))
print(type(soup.find(name='ul')))
print(soup.find(class_='list'))

<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>
<class 'bs4.element.Tag'>
<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>


其它方法：

```
find_parents 和 find_parent：前者返回所有祖先节点，后者返回直接父节点。

find_next_siblings 和 find_next_sibling：前者返回后面所有的兄弟节点，后者返回后面第一个兄弟节点。

find_previous_siblings 和 find_previous_sibling：前者返回前面所有的兄弟节点，后者返回前面第一个兄弟节点。

find_all_next 和 find_next：前者返回节点后所有符合条件的节点，后者返回第一个符合条件的节点。

find_all_previous 和 find_previous：前者返回节点前所有符合条件的节点，后者返回第一个符合条件的节点。
```

**CSS 选择器**`⭐⭐⭐`

Beautiful Soup 还提供 CSS 选择器。

使用 CSS 选择器，只需要调用 `select` 方法，传入相应的 CSS 选择器即可，用一个实例来感受一下：

In [25]:
html='''
<div class="panel">
    <div class="panel-heading">
        <h4>Hello</h4>
    </div>
    <div class="panel-body">
        <ul class="list" id="list-1">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
            <li class="element">Jay</li>
        </ul>
        <ul class="list list-small" id="list-2">
            <li class="element">Foo</li>
            <li class="element">Bar</li>
        </ul>
    </div>
</div>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
print(soup.select('.panel .panel-heading'))
print(soup.select('ul li'))
print(soup.select('#list-2 .element'))
print(type(soup.select('ul')[0]))

[<div class="panel-heading">
<h4>Hello</h4>
</div>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]
[<li class="element">Foo</li>, <li class="element">Bar</li>]
<class 'bs4.element.Tag'>


**嵌套选择**

`select` 方法同样支持嵌套选择，例如先选择所有 `ul` 节点，再遍历每个 `ul` 节点选择其 `li` 节点，样例如下：

In [26]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
for ul in soup.select('ul'):
    print(ul.select('li'))

[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>]
[<li class="element">Foo</li>, <li class="element">Bar</li>]


**获取属性**

由于节点类型是 Tag 类型，所以获取属性还可以用原来的方法。

仍然是上面的 HTML 文本，这里尝试获取每个 `ul` 节点的 `id` 属性：
```python
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
for ul in soup.select('ul'):
    print(ul['id'])
    print(ul.attrs['id'])
```

**获取文本**

可以用前面所讲的 string 属性。

此外，还可以使用 `get_text`方法，示例如下：
```python
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'lxml')
for li in soup.select('li'):
    print('Get Text:', li.get_text())
    print('String:', li.string)
```

**总结**

- 推荐使用 `lxml` 解析库，必要时使用 `html.parser`。
- 节点选择筛选功能弱但是速度快。
- 建议使用 `find、find_all` 方法查询匹配单个结果或者多个结果。
- 如果对 CSS 选择器熟悉的话可以使用 `select` 选择法。

<h3>使用 pyquery</h3>

解析库 pyquery 更适合使用 CSS 选择器。

像 `Beautiful Soup` 一样，初始化 `pyquery` 的时候，也需要传入 HTML 文本来初始化一个 PyQuery 对象。

它的初始化方式有多种，比如直接**传入字符串，传入 URL，传入文件名**，等等。

**字符串初始化**

In [27]:
html = '''
<div>
    <ul>
         <li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
'''
from pyquery import PyQuery as pq
doc = pq(html)
print(doc('li'))

<li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
         <li class="item

**URL 初始化**

初始化的参数不仅可以以字符串的形式传递，还可以传入网页的 URL，此时只需要指定参数为 url 即可：

In [36]:
from pyquery import PyQuery as pq
doc = pq(url='https://www.baidu.com', encoding='utf-8')
# 等价于 doc = pq(requests.get('https://www.baidu.com').text)
print(doc('title').text())

百度一下，你就知道


**文件初始化**

除了传递一个 URL，还可以传递本地的文件名，参数指定为 `filename` 即可：
> PyQuery 默认情况下会将 `<script>` 标签的内容包括在解析后的结果中。如果不想在输出中包含 `<script>` 标签的内容，你可以在使用 PyQuery 解析 HTML 后手动移除它们。

In [40]:
from pyquery import PyQuery as pq
doc = pq(filename='test.html')
doc('script').remove()
print(doc('li'))

<li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>

</body>
<!--human machine interact conversation-->
 
<!--自动弹出窗口 30s后自动关 单击跳转到另一页面-->
 
 
</html>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>

</body>
<!--human machine interact conversation-->
 
<!--自动弹出窗口 30s后自动关 单击跳转到另一页面-->
 
 
</html>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a

**基本 CSS 选择器**`⭐⭐⭐`

In [41]:
html = '''
<div id="container">
    <ul class="list">
         <li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
'''
from pyquery import PyQuery as pq
doc = pq(html)
# 先选取 id 为 container 的节点，然后再选取其内部的 class 为 list 的节点内部的所有 li 节点
print(doc('#container .list li'))
print(type(doc('#container .list li')))

<li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
         <li class="item

**查找子节点**

需要用到 `find` 方法，传入的参数是 CSS 选择器，我还是以上面的 HTML 为例：

In [47]:
from pyquery import PyQuery as pq

html = '''
<div id="container">
    <ul class="list">
         <li class="item-0">first item</li>
         <li class="item-1"><a href="link2.html">second item</a></li>
         <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
'''
doc = pq(html)
items = doc('.list')
# print(type(items))
# print(items)
# lis = items.find('li')
# print(type(lis))
# print(lis)

lis = items.children('.active')
print(lis)

<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
         <li class="item-1 active"><a href="link4.html">fourth item</a></li>
         <li class="item-0"><a href="link5.html">fifth item</a></li>
     </ul>
 </div>
         


**父节点**

用 `parent` 方法来获取某个节点的父节点，用一个实例来感受一下：
> 获取某个祖先节点，用`parents()`

In [48]:
html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''
from pyquery import PyQuery as pq
doc = pq(html)
items = doc('.list')
container = items.parent()
print(type(container))
print(container)

<class 'pyquery.pyquery.PyQuery'>
<div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
 


**兄弟节点**

使用 `siblings()` 方法。这里还是以上面的 HTML 代码为例：
```python
from pyquery import PyQuery as pq
doc = pq(html)
li = doc('.list .item-0.active')
print(li.siblings())
```

如果要筛选某个兄弟节点，依然可以向 siblings 方法传入 CSS 选择器，这样就会从所有兄弟节点中挑选出符合条件的节点了：
```python
from pyquery import PyQuery as pq
doc = pq(html)
li = doc('.list .item-0.active')
print(li.siblings('.active'))
```

**遍历**

刚才可以观察到，pyquery 的选择结果可能是多个节点，也可能是单个节点，类型都是 PyQuery 类型，并没有返回像 Beautiful Soup 那样的列表。

对于单个节点来说，可以直接打印输出，也可以直接转成字符串：
```python
from pyquery import PyQuery as pq
doc = pq(html)
li = doc('.item-0.active')
print(li)
print(str(li))
```

对于多个节点的结果就需要遍历来获取了。例如，这里把每一个 `li` 节点进行遍历，需要调用 `items` 方法：
```python
from pyquery import PyQuery as pq
doc = pq(html)
lis = doc('li').items()
print(type(lis))
for li in lis:
    print(li, type(li))
```

**获取信息**

提取到节点之后，我最终目的当然是提取节点所包含的信息了。

比较重要的信息有两类，一是**获取属性**，二是**获取文本**

**获取属性**

提取到某个 PyQuery 类型的节点后，就可以调用 `attr()` 方法来获取属性：

In [50]:
html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''
from pyquery import PyQuery as pq
doc = pq(html)
a = doc('.item-0.active a')
print(a, type(a))
print(a.attr('href'))

# 也可以通过调用 attr 属性来获取属性，用法如下
print(a.attr.href)

<a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div> <class 'pyquery.pyquery.PyQuery'>
link3.html
link3.html


**获取文本**

可以调用 `text` 方法来实现：

In [51]:
html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''
from pyquery import PyQuery as pq
doc = pq(html)
a = doc('.item-0.active a')
print(a)
print(a.text())

<a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
third item


**节点操作**

pyquery 提供了一系列方法来**对节点进行动态修改**，比如为某个节点添加一个 class，移除某个节点等，这些操作有时候会为提取信息带来极大的便利。

由于节点操作的方法太多，下面举几个典型的例子来说明它的用法。

**addClass 和 removeClass**

In [52]:
html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''
from pyquery import PyQuery as pq
doc = pq(html)
li = doc('.item-0.active')
print(li)
li.removeClass('active')
print(li)
li.addClass('active')
print(li)

<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
             
<li class="item-0"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
             
<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
             


**attr、text、html**

当然，除了操作 `class` 这个属性外，也可以用 `attr` 方法对属性进行操作。此外，还可以用 `text` 和 `html` 方法来改变节点内部的内容。示例如下：

In [53]:
html = '''
<ul class="list">
     <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
</ul>
'''
from pyquery import PyQuery as pq
doc = pq(html)
li = doc('.item-0.active')
print(li)
li.attr('name', 'link')
print(li)
li.text('changed item')
print(li)
li.html('<span>changed item</span>')
print(li)

<li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
</ul>

<li class="item-0 active" name="link"><a href="link3.html"><span class="bold">third item</span></a></li>
</ul>

<li class="item-0 active" name="link">changed item</li>
</ul>

<li class="item-0 active" name="link"><span>changed item</span></li>
</ul>



**remove 方法**

有时会为信息的提取带来非常大的便利。下面有一段 HTML 文本：

In [56]:
html = '''
<div class="wrap">
    Hello, World
    <p>This is a paragraph.</p>
 </div>
'''
from pyquery import PyQuery as pq
doc = pq(html)
wrap = doc('.wrap')
print(wrap.text())

# 提取 `Hello, World` 这个字符串，而不要 `p` 节点内部的字符串
wrap.find('p').remove()
print("------")
print(wrap.text())

Hello, World
This is a paragraph.
------
Hello, World


另外，其实还有很多节点操作的方法，比如 `append()、empty() 和 prepend()` 等方法，它们和 jQuery 的用法完全一致，

详细的用法可以参考官方文档：https://pyquery.readthedocs.io/en/latest/api.html

**伪类选择器**

CSS 选择器之所以强大，还有一个很重要的原因，那就是它支持多种多样的伪类选择器，

例如选择第一个节点、最后一个节点、奇偶数节点、包含某一文本的节点等。示例如下：

In [57]:
html = '''
<div class="wrap">
    <div id="container">
        <ul class="list">
             <li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
'''
from pyquery import PyQuery as pq
doc = pq(html)
li = doc('li:first-child')
print(li)
li = doc('li:last-child')
print(li)
li = doc('li:nth-child(2)')
print(li)
li = doc('li:gt(2)')
print(li)
li = doc('li:nth-child(2n)')
print(li)
li = doc('li:contains(second)')
print(li)

<li class="item-0">first item</li>
             <li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
             
<li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
         
<li class="item-1"><a href="link2.html">second item</a></li>
             <li class="item-0 active"><a href="link3.html"><span class="bold">third item</span></a></li>
             <li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href="link5.html">fifth item</a></li>
         </ul>
     </div>
 </div>
             
<li class="item-1 active"><a href="link4.html">fourth item</a></li>
             <li class="item-0"><a href

关于 CSS 选择器的更多用法，可以参考 https://www.w3school.com.cn/css/index.asp